## Project 4 Machine Learning Fairness Algorithms Evaluation

### LFR vs DM and DM-sen

Information systems are becoming increasingly reliant on statistical inference and learning to render all sorts
of decisions such as the targeting of advertising and the issuing of bank loans. With growing use of automated decision-making, problems rise. Unfairness arises around certain attributes, such as race and gender. Such attributes, some call "sensitive attributes", some call "protected group", often lead to unfair results based on attribute group. 

Today our group, group 2, is going to compare machine learning fairness algorithm performance that could handle sensitive attributes. 

Our target is LFR and DM and DM-sen, which represent Learning Fair Representation Algorithm and Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment Algorithm respectively. 

First let me introduce you with what each algorithm does.

### Learning fair representations (LFR)

LFR aims at achieving both group fairness (the proportion of members in a protected group receiving positive classification is identical to the proportion in the population as a whole), and individual fairness (similar individuals should be treated similarly). 

To achieve this, LFR formulate fairness as an optimization problem of finding a good representation of the data with two competing goals: 

1.encode data as well as possible

2.obfuscate information about membership in the protected group.

The method LFR uses is to create an intermediate representation of the original inputs that satisfy the above two goals. Then do further machine learning tasks based on this intermediate representation.

LFR maps each individual, represented as a data point in a given input space, to a probability distribution in a new representation space based on the following LOSS function from the paper.

<h3 align = "center">$Total \space Loss = A_z * L_z + A_x * L_x + A_y * L_y$<h3>

    where
<div align='center' ><font size='3'>$L_z= \sum_{k=1}^K|M_k^+-M_k^-|$</font></div>
<div align='center' ><font size='3'>$L_x = \sum_{n=1}^N (x_n - \hat{x}_n)^2$</font></div>
<div align='center' ><font size='3'>$L_y = \sum_{n=1}^N -y_n log \hat{y}_n - (1-y_n)log(1- \hat{y}_n)$</font></div> 

A_z, A_x, A_y are hyperparameters decide trade-offs between system desired data.

While the first two terms encourage the system to encode all information in the input attributes except for those that can lead to biased decisions, the third term requires that the prediction of y is as accurate as possible.

Data is first splitted based on protected and nonprotected. After doing individual train test split based on protected and nonprotected data, concate them together to make the final training, validation and test data. 

### Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment (DM and DM-sen)

DM and DM-sen introduced a new notion here, Disparate Mistreatment. As we all know, in order to maximize the utility of information systems (or, classifiers), their training involves minimizing the errors (or, misclassifications) over the given historical data. But it is quite possible that the optimally trained classifier makes decisions for people belonging to different social groups with different misclassification rates. This difference in misclassification rate is Disparate Mistreatment.

In order to measure disparate mistreatment, in the extent of decision boundary-based classifiers (e.g.logistic regression, SVMs), disparate mistreatment is measure using the covariance between the users’ sensitive attributes and the signed distance between the feature vectors of misclassified users and the classifier decision boundary. 

Then disparate mistreatment can be used in the constraints for minimizing the convex loss theta.

The final optimization problem is as following:

<h3 align = "center">$min\:\:L\left(\theta \right)$<h3>
<div align='center' ><font size='3'>$s.t.\:\frac{-N_1}{N}\sum _{\left(x,y\right)\in D_0}g_{\theta }\left(y,x\right)+\frac{N_0}{N}\sum \:_{\left(x,y\right)\in \:D_1}g_{\theta \:}\left(y,x\right)\le c$</font></div>
<div align='center' ><font size='3'>$\frac{-N_1}{N}\sum _{\left(x,y\right)\in D_0}g_{\theta }\left(y,x\right)+\frac{N_0}{N}\sum \:_{\left(x,y\right)\in \:D_1}g_{\theta \:}\left(y,x\right)\ge -c$</font></div>

Where $D_0$ and $D_1$ are the subsets of training dataset $D$ taking values $z = 0$ and $z = 1$, respectively, $N_0\:=\:\left|D_0\right|$ and $N_1\:=\:\left|D_1\right|$

### Model Performance and Comparison

In [5]:
import warnings
warnings.filterwarnings('ignore')
import os

In [16]:
%%capture
%run LFR.ipynb
%run ../lib/DM_DM_sen_Model.ipynb

#### LFR

In [21]:
print("priv error:", compute_error(y_hat_priv, test_y_plus))
print("non priv error:", compute_error(y_hat_nonpriv, test_y_minus))
print("overall error:", compute_error(np.concatenate((y_hat_nonpriv, y_hat_priv)), 
                                      np.concatenate((test_y_minus, test_y_plus))))

NameError: name 'y_hat_priv' is not defined

#### DM Logistic Regression

In [19]:
import warnings
warnings.filterwarnings('ignore')

loss_function = "logreg" 
EPS = 1e-6
cons_type = 0 # No constraint at very beginning 
tau = 5.0
mu = 1.2
sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution

cons_params = None
cons_params = {"cons_type": cons_type, "tau": tau, "mu": mu, "sensitive_attrs_to_cov_thresh": sensitive_attrs_to_cov_thresh}
start_dm = time.time()
return_accuracy_noConstraint()
return_accuracy_FPR()
return_accuracy_FNR()
return_accuracy_allConstraints()

end_dm = time.time()
runtime_dm = (end_dm-start_dm)
print(f'runtime of the complete DM model is {np.round(runtime_dm, 2)} seconds')

== Unconstrained (original) classifier ==
{'cons_type': 0, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.660
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.32 ||
||  1  || 0.18 || 0.62 ||




== Constraints on FPR ==
{'cons_type': 1, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.649
||  s  || FPR. || FNR. ||
||  0  || 0.27 || 0.41 ||
||  1  || 0.25 || 0.53 ||




== Constraints on FNR ==
{'cons_type': 2, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.651
||  s  || FPR. || FNR. ||
||  0  || 0.28 || 0.39 ||
||  1  || 0.29 || 0.47 ||




== Constraints on FNR and FPR ==
{'cons_type': 4, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.651
||  s  || FPR

#### DM Support vector machine (SVM)

In [20]:
import warnings
warnings.filterwarnings('ignore')

loss_function = "svm" 
EPS = 1e-6
cons_type = 0 # No constraint at very beginning 
tau = 5.0
mu = 1.2
sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution

cons_params = None
cons_params = {"cons_type": cons_type, "tau": tau, "mu": mu, "sensitive_attrs_to_cov_thresh": sensitive_attrs_to_cov_thresh}
start_dm = time.time()
return_accuracy_noConstraint()
return_accuracy_FPR()
return_accuracy_FNR()
return_accuracy_allConstraints()

end_dm = time.time()
runtime_dm = (end_dm-start_dm)
print(f'runtime of the complete DM model is {np.round(runtime_dm, 2)} seconds')

== Unconstrained (original) classifier ==
{'cons_type': 0, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.649
||  s  || FPR. || FNR. ||
||  0  || 0.38 || 0.30 ||
||  1  || 0.20 || 0.62 ||




== Constraints on FPR ==
{'cons_type': 1, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.650
||  s  || FPR. || FNR. ||
||  0  || 0.31 || 0.37 ||
||  1  || 0.27 || 0.51 ||




== Constraints on FNR ==
{'cons_type': 2, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.652
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.35 ||
||  1  || 0.27 || 0.50 ||




== Constraints on FNR and FPR ==
{'cons_type': 4, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.650
||  s  || FPR

### Result Interpretation

As we see, the accuracy for the LFR model is XX% and the accuracy of the DM and DM-sen model is XX%. There is an apparent decrease in accuracy from DM and DM-sen model to LFR model. This makes sense because fairness measure "cleans" the data to achieve fairness while DM and DM-sen does not remove information from the data.

But there could also be other explainations. There could be sampling bias. There might also be label bias where some mis-labeled entities fall into a sensitive group. Or maybe the ground truth(target) itself is just unfair, fairness measure might just harm model accuracy. There are more factors to consider.

In [13]:
pip install utils

Note: you may need to restart the kernel to use updated packages.
